## 지도 학습 기반 형태소 분석
- 품질 좋은 임베딩을 만들기 위해서는 문장이나 단어의 경계를 컴퓨터에 알려줘야 한다. **그렇지 않으면 어휘 집합에 속한 단어 수가 기하급수적으로 늘어나서 연산의 비효율이 발생**한다.
- 예를 들어 동사 '가다'의 활용형인 '가겠다', '가더라'도 모두 어휘 집합에 넣는다면 어휘 집합에 속한 단어 수는 총 2개가 된다. 이 방식이 문제는 새로운 활용형이 나타날 때마다 어휘 집합을 계속 늘려야한다는 점에 있다. '가겠더라', '갔다'라는 활용형도 말뭉치에 존재한다면 어휘 집합을 4개로 해야 한다. 이를 해결하기 위해 **형태소 분석 기법**을 사용한다. 

### KoNLPy 사용법
- 은전한닢, 꼬꼬마, 한나눔, Okt, 코모란 등 5개 오픈소스 형태소 분석기를 파이썬 환경에서 사용할 수 있도록 인터페이스를 통일한 한국어 자연어 처리 패키지 

In [ ]:
# 은전한닢 분석기 
from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs("아버지가방에들어가신다")

# 유의점: KoNLPy의 Mecab() 클래스는 Windows 환경에서 작동하지 않는다.

In [ ]:
# 은전한닢 품사 정보 확인
tokenizer.pos('아버지가방에들어가신다')

In [8]:
# get_tokenizer()
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

def get_tokenizer(tokenizer_name):
    if tokenizer_name == 'okt':
        tokenizer = Okt()
    if tokenizer_name == 'komoran':
        tokenizer = Komoran()
    if tokenizer_name == 'mecab':
        tokenizer = Mecab()
    if tokenizer_name == 'hannanum':
        tokenizer = Hannanum()
    if tokenizer_name == 'kkma':
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()  # default 
    return tokenizer 

# 코모란 사용 예시
tokenizer = get_tokenizer('komoran')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

['아버지', '가방', '에', '들어가', '시', 'ㄴ다']
[('아버지', 'NNG'), ('가방', 'NNP'), ('에', 'JKB'), ('들어가', 'VV'), ('시', 'EP'), ('ㄴ다', 'EC')]


#### KoNLPy 내 분석기별 성능 차이 
- 여러 가지 형태소 분석기의 성능을 비교해보면 은전한닢이 다른 분석기보다 속도가 빠른 것을 확인할 수 있다. 
- 속도만큼 중요한 것이 형태소 분석 품질이다. 

**어떤 형태소 분석기를 사용할지는 자신이 가진 데이터로 시험 삼아 형태소 분석을 해보고 속도나 품질을 비교해서 고르는 것이 좋다.** 

### Khaiii 사용법
- 카카오가 2018년 말 공개한 오픈소스 한국어 형태소 분석기
- 국립국어우너이 구축한 세종 코퍼스를 이용해 CNN 모델을 적용해 학습했다. 
- 아키텍처 개요: 입력문장을 문자 단위로 읽어 들임 -> 컨볼루션 필터가 문자들을 슬라이딩해 가면서 정보 추출 -> 네트워크의 말단 레이어에서 이렇게 모은 정보들을 종합해 형태소의 경계와 품사 태그를 예측 

In [ ]:
from khaiii import khaiiiApi
tokenizer = khaiiiApi()

In [ ]:
data = tokenizer.analyze('아버지가방에들어가신다')
tokens = []
for word in data:
    tokens.extend([str(m).split('/')[0] for m in word.morphs])

In [ ]:
# 품사 정보 확인 
data = tokenizer.analyze('아버지가방에들어가신다')
tokens = []
for word in data:
    tokens.extend([str(m) for m in word.morphs])

### 은전한닢에 사용자 사전 추가하기

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs('가우스전자 텔레비전 정말 좋네요')

관심 단어인 ``가우스전자``가 의도치 않게 2개의 토큰으로 분석된 것을 확인할 수 있다. ``가우스전자`` 하나로 분석됐을 때보다 데이터 분석이나 임베딩 품질이 떨어질 가능성이 적지 않다. 따라서 **관심 단어들을 사용자 사전에 추가**해 ``가우스전자``같은 단어가 하나의 토큰으로 분석될 수 있도록 강제해야 한다. 

In [ ]:
# 우선 /notebooks/embedding/preprocessmecab-user-dic.csv 파일에 붙여 썼으면 하는 단어를 추가하기 

# 은전한닢 사용자 사전 컴파일(bash)
# bash preprocess.sh mecab-user-dic 